In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'us_donations_2024.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.sample(5)

,Donor Name,Email,Phone,Donation Amount (USD),Donation Type,Project,Date,Location,Payment Method,Notes
1256,Taylor Lopez,lambheather@example.net,(999)706-2265x238,1146.44,Sadaqah,Feed the Hungry,2024-06-20,"Connerborough, Texas",Card,Whether court security young.
1534,Jennifer Flores,troy71@example.net,+1-796-240-2331,39958.70,Sadaqah,Medical Aid,2024-05-08,"East Vicki, Georgia",Apple Pay,Charge share expect less.
715,Barry Sanchez,wbrown@example.net,+1-280-917-8793,2742.70,Qurbani,Medical Aid,2024-06-14,"East Janice, Ohio",Bank Transfer,Responsibility film within support.
1305,Paul Jones,lopezrobert@example.net,(708)287-0158x92426,973.77,Zakat,Medical Aid,2024-03-07,"Port Tracystad, Illinois",PayPal,Hair you prepare scene drive executive first a...
23,Karen Garcia,lori14@example.net,905-339-1161x476,3931.42,Qurbani,Orphan Children,2024-01-28,"West Monicaburgh, Georgia",Card,Management television eat expect evening.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Donor Name             2500 non-null   object 
 1   Email                  2238 non-null   object 
 2   Phone                  2500 non-null   object 
 3   Donation Amount (USD)  2500 non-null   float64
 4   Donation Type          2500 non-null   object 
 5   Project                2500 non-null   object 
 6   Date                   2500 non-null   object 
 7   Location               2500 non-null   object 
 8   Payment Method         2500 non-null   object 
 9   Notes                  2500 non-null   object 
dtypes: float64(1), object(9)
memory usage: 195.4+ KB


In [6]:
currency = "BDT" if "bdt" in file_path.lower() or "bangladesh" in file_path.lower() else "USD"
amount_col = [col for col in df.columns if "donation amount" in col.lower()][0]

In [7]:
df['Email'] = df['Email'].fillna('No email provided').str.lower().str.strip()

In [8]:
df['Donation Type'] = df['Donation Type'].str.strip().str.capitalize()

In [9]:
df['Project'] = df['Project'].str.replace(r'\s+', ' ', regex=True).str.strip().str.title()

In [10]:
df['Project'].unique()

array(['Orphan Children', 'Feed The Hungry', 'Medical Aid',
       'Homeless Shelter', 'Back To School Kits',
       'Back To School Kits Want', 'Homeless Shelter Avoid',
       'Feed The Hungry Anything', 'Feed The Hungry Century',
       'Medical Aid Quality', 'Homeless Shelter Choice',
       'Homeless Shelter Present', 'Orphan Children Go',
       'Homeless Shelter War', 'Feed The Hungry Home',
       'Homeless Shelter Whole', 'Back To School Kits Morning',
       'Medical Aid Chance', 'Medical Aid Near', 'Medical Aid Score',
       'Homeless Shelter Right', 'Orphan Children Result',
       'Back To School Kits Technology', 'Medical Aid Interesting',
       'Homeless Shelter Writer', 'Homeless Shelter Join',
       'Orphan Children Around', 'Homeless Shelter Actually',
       'Homeless Shelter Provide', 'Homeless Shelter Military',
       'Homeless Shelter Song', 'Homeless Shelter Gas',
       'Medical Aid Even', 'Medical Aid Tough', 'Homeless Shelter Simple',
       'Orphan Childr

In [11]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.3 MB/s eta 0:00:00


In [12]:
from rapidfuzz import process

valid_projects = ['Orphan Children', 'Feed the Hungry', 'Homeless Shelter', 'Medical Aid', 'Back to School Kits', 'Rohingya Refugee Support', 'Medical Care', 'Eid Gifts For Children']

def match_project(proj_name):
    match, score, _ = process.extractOne(proj_name, valid_projects)
    return match if score > 80 else proj_name

df['Project'] = df['Project'].apply(match_project)

In [13]:
df['Project'].unique()

array(['Orphan Children', 'Feed the Hungry', 'Medical Aid',
       'Homeless Shelter', 'Back to School Kits'], dtype=object)

In [14]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
# df['Day'] = df['Date'].dt.day

In [15]:
df = df.drop_duplicates()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2381 entries, 0 to 2380
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Donor Name             2381 non-null   object        
 1   Email                  2381 non-null   object        
 2   Phone                  2381 non-null   object        
 3   Donation Amount (USD)  2381 non-null   float64       
 4   Donation Type          2381 non-null   object        
 5   Project                2381 non-null   object        
 6   Date                   2381 non-null   datetime64[ns]
 7   Location               2381 non-null   object        
 8   Payment Method         2381 non-null   object        
 9   Notes                  2381 non-null   object        
 10  Month                  2381 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(8)
memory usage: 213.9+ KB


In [17]:
threshold = df[amount_col].quantile(0.95)
df = df[df[amount_col] <= threshold]

In [18]:
df['Phone'] = df['Phone'].astype(str).str.replace(r'\D', '', regex=True)
df['Phone'] = df['Phone'].apply(lambda x: f"+880{x[-10:]}" if currency == "BDT" else f"+1{x[-10:]}" if len(x) >= 10 else "Invalid")

In [19]:
df['Payment Method'] = df['Payment Method'].str.strip().str.title()
if currency == "BDT":
    allowed_pm = ['Bkash', 'Nagad', 'Bank Transfer', 'Card']
else:
    allowed_pm = ['Card', 'Bank Transfer', 'Paypal', 'Apple Pay']
df['Payment Method'] = df['Payment Method'].apply(
    lambda x: x if x in allowed_pm else 'Other'
)

In [20]:
df['Notes'] = df['Notes'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

In [21]:
df['Location'] = df['Location'].str.title().str.strip()

In [22]:
df.rename(columns={amount_col: f'Donation Amount ({currency})'}, inplace=True)

In [23]:
df.head()

,Donor Name,Email,Phone,Donation Amount (USD),Donation Type,Project,Date,Location,Payment Method,Notes,Month
0,Grace Foster,carloconnell@example.com,+13989717117,3493.24,Zakat,Orphan Children,2024-06-05,"New Rachel, North Carolina",Bank Transfer,Before method edge hand.,6
1,Seth Jackson,george37@example.org,+18857124596,806.16,Sadaqah,Feed the Hungry,2024-12-22,"Carterchester, California",Bank Transfer,Face final month college improve analysis seve...,12
2,Kimberly Morrison,cynthiali@example.com,+13306537445,2176.76,Fidya,Medical Aid,2024-10-13,"Princeborough, Georgia",Apple Pay,Her dream feeling player.,10
3,Michael Davis,jeffrey04@example.net,+19357583995,1510.32,Fidya,Orphan Children,2024-03-11,"South Thomas, New York",Paypal,Sing know newspaper.,3
4,Michael Rojas,wrightjustin@example.org,+12482146024,4246.96,Qurbani,Homeless Shelter,2024-06-07,"New Joseph, California",Bank Transfer,Mr dog simple laugh especially during mention.,6


In [24]:
df.to_csv(f'cleaned_{file_path}', index=False)